<a href="https://colab.research.google.com/github/Staneering/School-of-Data-24/blob/main/customer_sentiment_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Loading Dataset

In [1]:
import zipfile

with zipfile.ZipFile(r'/content/customer_data.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_data')

In [5]:
customer = pd.read_csv(r'/content/unzipped_data/customers.csv')
event = pd.read_csv(r'/content/unzipped_data/events.csv')
order = pd.read_csv(r'/content/unzipped_data/orders.csv')
items = pd.read_csv(r'/content/unzipped_data/line_items.csv')
product = pd.read_csv(r'/content/unzipped_data/products.csv')

In [6]:
product.head()
#customer.info()
#customer.describe()

,id,name,price
0,1,iPhone 13,1099.99
1,2,Samsung Galaxy S21,899.99
2,3,Sony PlayStation 5,499.99
3,4,Microsoft Xbox Series X,499.99
4,5,Nike Air Max 270,129.99


## Merging Dataset

In [7]:
merged_data = pd.merge(customer,event, on='customer_id', how='inner')
merged_data = pd.merge(merged_data,order, on='customer_id',how='inner')
merged_data = pd.merge(merged_data, items, on='order_id', how='inner')
merged_data = pd.merge(merged_data, product, left_on='item_id', right_on='id', how='inner')

## Handling Missing Values

In [8]:
merged_data.isnull().sum()

,0
customer_id,0
device_id,0
location,0
currency,0
event_id,0
event_data,0
event_timestamp,0
order_id,0
status,0
checked_out_at,0


In [9]:
merged_data.columns = [col.lower().strip().replace(' ', '_') for col in merged_data.columns]

In [10]:
merged_data.shape

(578320, 16)

In [11]:
merged_data.head()

,customer_id,device_id,location,currency,event_id,event_data,event_timestamp,order_id,status,checked_out_at,line_item_id,item_id,quantity,id,name,price
0,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3201,4,2,4,Microsoft Xbox Series X,499.99
1,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3202,5,4,5,Nike Air Max 270,129.99
2,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3203,9,5,9,Nintendo Switch,299.99
3,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3204,12,1,12,Coca-Cola 12-Pack,5.99
4,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3205,6,2,6,Adidas Ultraboost,180.00


In [15]:
merged_data['event_data'].head()

,event_data
0,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e..."
1,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e..."
2,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e..."
3,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e..."
4,"{""timestamp"": ""2024-03-26T22:05:58.863661"", ""e..."


In [24]:
#unpacking eventdata
merged_data['event_data'].iloc[0]


'{"timestamp": "2024-03-26T22:05:58.863661", "event_type": "visit"}'

In [25]:
#convert the string to a dictionary
#event_dict =eval(merged_data['event_data'].iloc[0])
#type(event_dict)


merged_data['event_data'] = merged_data['event_data'].apply(eval)
merged_data['event_data'].iloc[0]

{'timestamp': '2024-03-26T22:05:58.863661', 'event_type': 'visit'}

In [27]:
#extract event type
#merged_data['event_data'].iloc[0]['event_type']


#create a new column called event type
merged_data['event_type']=merged_data['event_data'].apply(lambda x: x['event_type'])

In [28]:
merged_data['event_type'].head()

,event_type
0,visit
1,visit
2,visit
3,visit
4,visit


In [29]:
#checking for the unique values in the event type column
merged_data['event_type'].unique()

array(['visit', 'add_to_cart', 'remove_from_cart', 'checkout'],
      dtype=object)

In [30]:
merged_data.head()

,customer_id,device_id,location,currency,event_id,event_data,event_timestamp,order_id,status,checked_out_at,line_item_id,item_id,quantity,id,name,price,event_type
0,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{'timestamp': '2024-03-26T22:05:58.863661', 'e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3201,4,2,4,Microsoft Xbox Series X,499.99,visit
1,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{'timestamp': '2024-03-26T22:05:58.863661', 'e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3202,5,4,5,Nike Air Max 270,129.99,visit
2,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{'timestamp': '2024-03-26T22:05:58.863661', 'e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3203,9,5,9,Nintendo Switch,299.99,visit
3,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{'timestamp': '2024-03-26T22:05:58.863661', 'e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3204,12,1,12,Coca-Cola 12-Pack,5.99,visit
4,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,"{'timestamp': '2024-03-26T22:05:58.863661', 'e...",2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3205,6,2,6,Adidas Ultraboost,180.00,visit


In [ ]:
merged_data.drop_duplicates(inplace=True)

In [33]:
#dropping redundant columns:event data,id and item id)
merged_data.drop(columns=['id','item_id','event_data'],inplace = True)


KeyError: "['id', 'item_id', 'event_data'] not found in axis"

In [34]:
merged_data.head()

,customer_id,device_id,location,currency,event_id,event_timestamp,order_id,status,checked_out_at,line_item_id,quantity,name,price,event_type
0,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3201,2,Microsoft Xbox Series X,499.99,visit
1,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3202,4,Nike Air Max 270,129.99,visit
2,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3203,5,Nintendo Switch,299.99,visit
3,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3204,1,Coca-Cola 12-Pack,5.99,visit
4,cda7bec0-7cbf-4145-baf9-e049bab94504,33b485de-7338-4997-b1d0-b988ba17b245,Saint Helena,NGN,14786,2024-03-26 22:05:58.863661,9e950ce3-2e12-479c-a051-493c1a497d49,success,2024-03-30 04:19:10.406794,3205,2,Adidas Ultraboost,180.00,visit


In [ ]:
merged_data.to_csv('merged_data.csv', index=False)

## Questions
##  a. Visit Frequency:
## How often does this customer visit the website, based on the provided data?
## Can you identify any patterns or trends in the customer's visit frequency?
## b. Location Analysis:
## What is the customer's location based on the provided data?
## How might the customer's location influence their purchasing behavior?
## c. Overall Purchase Behavior:
## Based on the data provided, what insights can you draw about this customer's overall behavior on the website?
## How might these insights inform marketing strategies or personalized recommendations for this customer?



In [ ]:
merged_data['event_timestamp'] = pd.to_datetime(merged_data['event_timestamp'])

In [ ]:
visit_frequency = merged_data.groupby('customer_id')['event_timestamp'].count().reset_index()
visit_frequency.rename(columns={'event_timestamp': 'visit_frequency'}, inplace=True)
merged_data = pd.merge(merged_data, visit_frequency, on='customer_id', how='left')
visit_frequency

,customer_id,visit_frequency
0,00096d0c-ced3-4913-afe9-5ce363f7126f,253
1,000c7914-248f-404c-bd49-49c7ae49364e,275
2,000d9458-f109-4fd1-9bc9-2514dde6c7d2,288
3,000e74fd-2f7f-447c-9fe5-a0e856e3e93f,12
4,001c37e2-f25c-45c1-8326-1ddf73c21dde,42
...,...,...
4589,ffaf35ad-37fa-43f1-bed3-26bb53e612ae,133
4590,ffc51ce7-426f-4e93-b98e-21be2202b40e,200
4591,ffda8205-bed4-4067-b4f6-8ed1b4819c64,78
4592,ffddb31d-9c3e-46b7-9ec2-efd272f6bd51,98


In [ ]:
merged_data['last_visit'] = merged_data.groupby('customer_id')['event_timestamp'].transform('max')
today = pd.Timestamp.now()
merged_data['recency'] = (today - merged_data['last_visit']).dt.days

In [36]:
merged_data['location'].value_counts()

,count
location,
Korea,6997
Argentina,4507
Mauritius,4414
Jersey,4130
Panama,3936
...,...
Chad,1082
Malaysia,1037
Austria,921


In [ ]:
preferred_category = merged_data.groupby('customer_id')['name'].agg(lambda x: x.value_counts().idxmax()).reset_index()
preferred_category.rename(columns={'name': 'preferred_category'}, inplace=True)
merged_data = pd.merge(merged_data, preferred_category, on='customer_id', how='left')
preferred_category

,customer_id,preferred_category
0,00096d0c-ced3-4913-afe9-5ce363f7126f,Coca-Cola 12-Pack
1,000c7914-248f-404c-bd49-49c7ae49364e,Nintendo Switch
2,000d9458-f109-4fd1-9bc9-2514dde6c7d2,Samsung Galaxy S21
3,000e74fd-2f7f-447c-9fe5-a0e856e3e93f,Dell XPS 13 Laptop
4,001c37e2-f25c-45c1-8326-1ddf73c21dde,Nintendo Switch
...,...,...
4589,ffaf35ad-37fa-43f1-bed3-26bb53e612ae,Adidas Ultraboost
4590,ffc51ce7-426f-4e93-b98e-21be2202b40e,Adidas Ultraboost
4591,ffda8205-bed4-4067-b4f6-8ed1b4819c64,Starbucks Coffee Beans
4592,ffddb31d-9c3e-46b7-9ec2-efd272f6bd51,iPhone 13


In [ ]:
# 5. Location-Based Insights
location_data = merged_data.groupby('location')['price'].sum().reset_index()
location_data.rename(columns={'price': 'location_total'}, inplace=True)
merged_data = pd.merge(merged_data, location_data, on='location', how='left')
location_data

,location,location_total
0,Afghanistan,1599882.86
1,Albania,2616473.17
2,Algeria,1201012.33
3,American Samoa,1168791.48
4,Andorra,2020906.52
...,...,...
238,Wallis and Futuna,1217759.26
239,Western Sahara,1454831.54
240,Yemen,1005931.74
241,Zambia,1213199.25


In [37]:
merged_data.status.unique()

array(['success', 'failed', 'cancelled'], dtype=object)